#### imports

In [2]:
from google.colab import drive
drive.mount('/content/drive') # pour monter notre drive au notebook

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import librosa
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import pickle

# get spectrograms as arrays
✅functions to use

In [4]:
def get_mel_spec_array(y,sr):
  ''' 
  Parameters :
    y : audio data
    sr : sampling rate

  Returns :
    S_DB : np.array of the mel spectrogram (log y scale)
  '''

  S = librosa.feature.melspectrogram(y=y, sr=sr) # Compute a mel-scaled spectrogram
  S_DB = librosa.power_to_db(S, ref=np.max) # Convert a power spectrogram (amplitude squared) to decibel (dB) units

  return S_DB

In [5]:
def get_lin_spec_array(y,sr):
  ''' 
  Parameters :
    y : audio data
    sr : sampling rate

  Returns :
    np.array of the spectrogram (linear scale)
  '''
  # TODO
  return None

In [6]:
def get_log_spec_array(y,sr):
  ''' 
  Parameters :
    y : audio data
    sr : sampling rate

  Returns :
    np.array of the spectrogram (logarithmic scale)
  '''
  # TODO
  return None

# get family or species code  
⛔ not used

In [ ]:
# The following funtion is not used since we always want to return the species_code, not the family code
def get_target_code(species_code,target):
  """
  Parameters:
    species_code : species code following the watkins database classification
    target : 'family' or 'species'
  
  Returns:
    the family code or the species code (string)
  """
  if target == 'family':
    return species_code[:2]

  if target == 'species':
    return species_code


# Function to get the tuple (array, species_code) for **one** audio file  
⛔ not used

In [ ]:
def get_array_and_species_code(audio_directory, filename, species_code, spectro_type='mel', sr=44_100):
  '''
  Load an audio file and return a tuple with spectrogram array and species code

  Parameters
    audio_directory : path to the directory where audio is stored
    filename : name of the audio file
    species_code : code of the species recorded on the audio file
    spectro_type : type of spectrogram used ('mel', 'log', 'lin')
    sr : sampling rate

  Returns
    array : np.array of the mel spectrogram (log y scale)
    species_code : the species code
  '''
  
  # path to audio file (audio_directory and filename)
  filepath = os.path.join(audio_directory,filename)

  # load the audio file
  y, sr = librosa.load(filepath,sr=sr)

  # compute the correct spectrogram
  if spectro_type == 'mel':
    array = get_mel_spec_array(y,sr)
  elif spectro_type == 'log':
    array = get_log_spec_array(y,sr)
  elif spectro_type == 'lin':
    array = get_lin_spec_array(y,sr)
  else:
    array=None

  return array, species_code

# Function to get the list of tuples (array, species_code) for **all** audio files  
⛔ not used

In [ ]:
def get_arrays_and_species_codes(csv_path, audio_directory, sr=44_100, spectro_type='mel', nb_rows=None):
  '''
  Takes a CSV file containing the names of the sound files to be treated.
  Returns a list where each item is a tuple of two items:
    - the array of the spectrogram
    - the corresponding species code

  Parameters:
    csv_path : path to the csv file with all filenames and information
    audio_directory : path to the directory where audio is stored
    sr : sampling rate (default = 44100 Hz)
    spectro_type : type of spectrogram used ('mel', 'log', 'lin')
    nb_rows : number of rows to iterate over in the csv (to allow testing on small number of rows)

  Returns:
    list_of_tuples :  a list of tuples containing the array and the species code
  '''

  # load the csv into a dataframe
  df = pd.read_csv(csv_path)

  # resize the df if requested in parameters
  if nb_rows:
    df = df.head(nb_rows)

  # iterate over the rows of the df to get the array and the species code for each audio file
  list_of_tuples = df.apply(lambda row: get_array_and_species_code(audio_directory, row.filename, row.species_code, spectro_type, sr),axis=1).tolist()
  
  return list_of_tuples


# Function to get arrays from audio_data
✅ function to use

In [1]:
def get_arrays_from_audio(input_list, spectro_type='mel', sr = 44_100, target='family'):
  '''
  Takes a list of tuples (audio_data,species_code).
  Returns a list of tuples (array,species_code)

  Parameters:
    input_list : list of tuples (audio_data,species)
    sr = sample rate. Default: 44100 Hz
    target: 'family' or 'species'

  Returns:
    list_of_tuples :  list of tuples (array,species)
  '''

  list_of_tuples = []

  for input_tuple in input_list:
    my_list = []

    # compute the correct spectrogram
    if spectro_type == 'mel':
      array = get_mel_spec_array(input_tuple[0],sr)
    elif spectro_type == 'log':
      array = get_log_spec_array(input_tuple[0],sr)
    elif spectro_type == 'lin':
      array = get_lin_spec_array(input_tuple[0],sr)
    else:
      array=None
    
    # append the array
    my_list.append(array)

    # append the target (family or species)
    if target == 'family':
      my_list.append(input_tuple[1][:2])
    if target == 'species':
      my_list.append(input_tuple[1])
    
    # convert the list in a tuple
    my_tuple = tuple(my_list)

    # append the tuple to the output list
    list_of_tuples.append(my_list)
  
  return list_of_tuples

# Test

In [ ]:
audio_directory = '/content/drive/MyDrive/lewagon-deepdive/raw_data/wav_files'
csv_path = '/content/drive/MyDrive/lewagon-deepdive/raw_data/clean_dataset_1.csv'
sr = 44_100
spectro_type = 'mel'
nb_rows = 3 # to be set as None if you want to iterate the whole dataset

In [ ]:
results = get_arrays_and_species_codes(csv_path, audio_directory, sr=sr, spectro_type='mel', nb_rows=nb_rows)

In [ ]:
results[1]

(array([[-32.650093, -34.818665, -35.641525, ..., -41.056828, -41.261032,
         -35.64364 ],
        [-40.283695, -36.565804, -38.450073, ..., -35.811836, -37.9296  ,
         -34.138645],
        [-32.645184, -35.637314, -37.25069 , ..., -38.254654, -43.363087,
         -38.868404],
        ...,
        [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
         -80.      ],
        [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
         -80.      ],
        [-80.      , -80.      , -80.      , ..., -80.      , -80.      ,
         -80.      ]], dtype=float32), 'AA1A')

# Using a pickle file

In [ ]:
pickle_name = 'test_pickle.pkl'
pickle_location = '/content/drive/MyDrive/lewagon-deepdive/working_environment/01.getting_data'

In [ ]:
with open(f'{pickle_location}/{pickle_name}', 'wb') as f:
  pickle.dump(results, f)

In [ ]:
my_list = pickle.load(open(f'{pickle_location}/{pickle_name}', 'rb'))
len(my_list)

3

# Separate the tuples in two different lists

In [ ]:
array_list = []
species_code_list = []

for (array, species_code) in list_of_tuples:
  array_list.append(array)
  species_code_list.append(species_code)